In [1]:
def imshow(a):
    try:
        a = cv2.normalize(a, None, alpha = 0, beta = 255, norm_type = cv2.NORM_MINMAX, dtype = cv2.CV_32F)
        a = a.astype(np.uint8)
    except:
        pass
    display(Image.fromarray(a))

In [2]:
import glob
import os
from pathlib import Path
import numpy as np
from dask import array as da
import random
import tifffile
import cv2
from PIL import Image
import warnings

In [3]:
def load_data(path):
    grid = np.loadtxt(path, skiprows=6)
#     visualization = cv2.imread(f"{path[5:-4]}.png", cv2.IMREAD_COLOR)
    tif_paths = glob.glob(f"data/{path[5:-4]}/download??.tif")
    try:
        image = da.dstack([tifffile.memmap(tif) for tif in tif_paths])
        print("memmap successful")
    except ValueError:
        if tif_paths:
            image = np.dstack([tifffile.imread(tif) for tif in tif_paths])
        else:
            warnings.warn(f"Image files not found for {path}, empty (white) canvas will be returned.")
            return grid, np.full((grid.shape[0], grid.shape[1], 3), 255, dtype=np.uint8)
    grid = cv2.resize(grid, dsize=image.shape[:-1][::-1], interpolation=cv2.INTER_AREA)
    return grid, image

In [4]:
def blockshaped(arr, area):
    """
    Return an array of shape (n, area, area) where
    n * area * area = arr.size

    If arr is a 2D array, the returned array should look like n subblocks with
    each subblock preserving the "physical" layout of arr.
    """
    h, w = arr.shape[:2]
    h_cut = h - h%area
    w_cut = w - w%area
    out = arr[:h_cut, :w_cut]
#         out = np.pad(arr, ((0, area-h%area), (0, area-w%area)), 'constant', constant_values=-999)
    try:
        h, w = out.shape
        return (out.reshape(h//area, area, w//area, area)
               .swapaxes(1,2)
               .reshape(-1, area, area))
    except ValueError:
#         out = cv2.copyMakeBorder(arr, 0, area-h%area, 0, area-w%area, borderType=cv2.BORDER_CONSTANT,value=[0,0,0])
        h, w, c = out.shape
        return (out.reshape(h//area, area, w//area, area, c)
               .swapaxes(1,2)
               .reshape(-1, area, area, c))

In [5]:
def partition_and_filter(grid, image, area):
    subgrids = blockshaped(grid, area)
    del grid
    subimages = blockshaped(image, area)
    del image
    mask = (subgrids > 9999).any(axis=(1, 2)) | (subgrids < -99).any(axis=(1, 2))
    print(f"Damaged subgrids:\t{mask.sum()}")
    subgrids = subgrids[~mask]
    subimages = subimages[~mask]
    
    np.save('temp/grids.npy', subgrids)
    del subgrids
    
    print("Loading satellite image...", end="")
    subimages = np.asarray(subimages)
    print("\tdone!")
    mask = (subimages == (0, 0, 0)).any(axis=(1, 2, 3))
    print(f"Damaged subimages:\t{mask.sum()}")
    subimages = subimages[~mask]
    
    subgrids = np.load('temp/grids.npy')
    subgrids = subgrids[~mask]
    return subgrids, subimages

In [6]:
def train_val_test_split(grid, image, train=0.8, val=0.1, test=0.1):
    assert train + val + test == 1.0
    assert grid.shape == image.shape[:3]
    indices = np.arange(grid.shape[0])
    np.random.shuffle(indices)
    grid = grid[indices]
    image = image[indices]
    train_grid, validate_grid, test_grid = np.split(grid, [int(len(grid)*train), int(len(grid)*(train+val))])
    del grid
    train_images, validate_images, test_images = np.split(image, [int(len(image)*train), int(len(image)*(train+val))])
    del image
    return train_grid, validate_grid, test_grid, train_images, validate_images, test_images

In [7]:
def main():
    paths = glob.glob(f"data/*.asc")
    for path_in in paths:
        print(f"loading {path_in}...", end='')
        path_out = f'out/{path_in[5:-4]}/'
        path_out = f"D:/out_shuffle/{path_in[5:-4]}/"
        if os.path.exists(path_out):
            print("skipping...")
            continue
        grid, image = load_data(path_in)
        Path(path_out).mkdir(parents=True, exist_ok=True)
        
        print("done!\npartitioning the data...")
        subgrids, subimages = partition_and_filter(grid, image, 256)
        del grid, image
        
        print(f"done!\ttotal subareas:\t{len(subgrids)}\nsplitting the data...")
        train_grid, validation_grid, test_grid, train_images, validation_images, test_images = train_val_test_split(subgrids, subimages)
        del subgrids, subimages
        
        print("done!\nsaving the data...")
        np.save(path_out+'train_grids.npy', train_grid)
        np.save(path_out+'train_images.npy', train_images)
        del train_grid, train_images
        np.save(path_out+'test_grids.npy', test_grid)
        np.save(path_out+'test_images.npy', test_images)
        del test_grid, test_images
        np.save(path_out+'validation_grids.npy', validation_grid)
        np.save(path_out+'validation_images.npy', validation_images)
        del validation_grid, validation_images
        
        print("done!")
    return

In [8]:
main()

loading data\death_valley.asc...skipping...
loading data\laytonville.asc...skipping...
loading data\mt_rainier.asc...done!
partitioning the data...
Damaged subgrids:	2032
Loading satellite image...	done!
Damaged subimages:	0
done!	total subareas:	2478
splitting the data...
done!
saving the data...
done!
loading data\river_basin.asc...skipping...
loading data\san_gabriel.asc...skipping...
